# Col de Porte Observations

In [1]:
# netcdf/numpy/xray/stats
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
from scipy.stats.stats import pearsonr

# OS interaction
import sys, pickle, os

# import plotting
import seaborn as sns
import matplotlib
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap

# Offline Turbulence Package
import turbpy

# Customize
sns.set_style("whitegrid")
sns.set_context('paper')
%matplotlib inline

In [2]:
# --------------------------------------------------------------------------------------------------------------------
# Directory Lists
# Unix
if 'linux' in sys.platform:
    dirPre = '/home/lapok/gdrive/'
# Mac
elif 'darwin' in sys.platform:
    dirPre = '/Users/karllapo/gdrive/'

dirProj = dirPre + 'SnowHydrology/proj/ModTsfc/'
dirPrint = dirProj + 'Graphics'
dirData = dirProj + 'data'

# Original Snoqualmie observations
dirDataRaw = dirProj + 'data/CDP'

# Sub-functions for calculating dewpoint/RH

In [3]:
def calc_RH(p,q,T):
    RH = np.empty_like(T)
    if np.nanmin(T) < 200:
        T = T + 273.16
    T_0 = 273.16

    RH = .263*p*q* (np.exp( (17.67*(T-T_0)) / (T-29.65) ))**(-1)
    RH[RH > 100] = 100
    return(RH)

def calc_Tdew(T,RH):
    # Unit checks
    if np.nanmax(T) > 100 or np.nanmin(T) > 40:
        raise ValueError('Air temperature must be in Celsius')

    if np.nanmax(RH) > 1 or np.nanmin(RH) < 0:
        RH = RH/100
        if np.nanmax(RH) > 1 or np.nanmin(RH) < 0:
            raise ValueError('Relative humidity must be a fraction on [0,1]')

    if not np.size(RH) == np.size(T):
        raise ValueError('Relative humidity and air temperature must have the same number of elements')

    # -----------------------------------------------------------------------------------------------
    # ALGORITHM
    # When are we calculating with respect to frost or water?
    frost_ind = np.flatnonzero(T <= 0)
    water_ind = np.flatnonzero(T > 0)
    # Frost coefficients 
    b_frost = 22.587
    c_frost = 273.86
    # Water coefficients
    b_water = 17.625
    c_water = 243.03

    # Pre-allocate
    Tdew = np.empty_like(T)

    Tdew[frost_ind] = MagnusTetens(T[frost_ind],RH[frost_ind],b_frost,c_frost)
    Tdew[water_ind] = MagnusTetens(T[water_ind],RH[water_ind],b_water,c_water)

    return(Tdew)

# SUB-FUNCTION for actual expression
def MagnusTetens(T,RH,b,c):
    dew = (c*( np.log(RH) + (b * T)/(c + T) )) / ( b - np.log(RH) - (b * T)/(c + T) )
    return(dew)

## CDP Model Forcing Data

In [4]:
os.chdir(dirDataRaw)
CDP = xr.open_dataset('CDP_met_insitu.nc')
print(CDP)

<xarray.Dataset>
Dimensions:          (location: 1, time: 157776)
Coordinates:
  * time             (time) datetime64[ns] 1993-08-01 ...
Dimensions without coordinates: location
Data variables:
    lat              (location) float64 45.3
    lon              (location) float64 5.77
    altitude         (location) float64 1.325e+03
    aspect           (location) float64 0.0
    slope            (location) float64 0.0
    ZREF             (location) float64 1.5
    UREF             (location) float64 10.0
    FORC_TIME_STEP   float64 3.6e+03
    Tair             (time, location) float64 282.4 282.4 282.3 282.2 282.1 ...
    Qair             (time, location) float64 0.006398 0.006309 0.006256 ...
    Wind_DIR         (time, location) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Wind             (time, location) float64 1.433 1.233 1.034 0.835 0.736 ...
    Rainf            (time, location) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Snowf            (time, location) float64 0.0 0.0 0.0 0

In [4]:
# date parsing function for pandas csv_read
def parse(timeStr):
    s = '00'
    date_str = timeStr + ':' + s
    dt = datetime.strptime(date_str,"%Y-%m-%dT%H:%M:%S")
    return dt

# -------------------------------------------------------------------------
# Read forcing data
# 24 rows of comments
os.chdir(dirDataRaw)
datafile = 'CDP_met_insitu.tab'
CDP = pd.read_csv(datafile,
                  sep='\t',
                  header=0,
                  skiprows=24,
                  date_parser=parse,
                  parse_dates={'time': [0]},
                  index_col='time'
                 )

# Use my naming convention
CDP.columns = ['Tair', 'QS', 'WIND', 'precipRain', 'precipSnow', 'LWdwn', 'SWdwnDir', 'SWdwnDif', 'Press']

# -------------------------------------------------------------------------
# Conversions
CDP['SWdwn'] = CDP['SWdwnDir'] + CDP['SWdwnDif']
CDP['precip'] = CDP['precipSnow'] + CDP['precipRain']
CDP['Press'] = CDP['Press'] * 100
CDP['QS'] = CDP['QS'] / 1000

# -------------------------------------------------------------------------
# RH - this needs some attention to units
CDP['RH'] = calc_RH(CDP.Press.values, CDP.QS.values, CDP.Tair.values)
# Dew point temperature
CDP['Tdew'] = calc_Tdew(CDP.Tair.values, CDP.RH.values)

# -------------------------------------------------------------------------
# Convert to xarray Dataset
CDP = xr.Dataset.from_dataframe(CDP)

## Read supporting evaluation data

In [5]:
# -------------------------------------------------------------------------
# Read supporting data
# 19 rows of comments
os.chdir(dirDataRaw)
datafile = 'CDP_hor_eval.tab'
support = pd.read_csv(datafile,
                  sep='\t',
                  header=0,
                  skiprows=20,
                  date_parser=parse,
                  parse_dates={'time': [0]},
                  index_col='time'
                 )

# Use my naming convention
support.columns = ['snowDepth', 'Tsrf', 'Lysimeter1', 'Lysimeter2',
               'groundHeatFlux1', 'groundHeatFlux2', 'groundHeatFlux3', 'Albedo']
support.drop(['Lysimeter1', 'Lysimeter2', 'groundHeatFlux1', 'groundHeatFlux2', 'groundHeatFlux3'],
             axis=1, inplace=True)

# -------------------------------------------------------------------------
# Convert to xarray Dataset
support = xr.Dataset(support)

# Take daily average, reindex to half-hourly time series, use in snow presence criteria
TsrfDaily = support.Tsrf.resample(how='mean', freq='d', dim='time', label='left')
TsrfDaily = TsrfDaily.reindex_like(support, method='ffill')

# bare ground when no snowdepth recorded or the daily surface temperature is above freezing
groundSurfTemp = support.Tsrf[(support.snowDepth == 0) | (TsrfDaily > 0.5)]

# snow covered ground when snow is observed and the surface temperature is below freezing
snowSurfTemp = support.Tsrf[(support.snowDepth > 0) & (TsrfDaily < 0.5)]

# Assign to support xarray.Dataset
support['groundTs'] = groundSurfTemp
support['snowTs'] = snowSurfTemp

# Create snow presence variable
snowPres = ((support.snowDepth > 0) | (TsrfDaily < 0.5))
support['SP'] = snowPres.astype(int)

//anaconda/lib/python3.5/site-packages/xarray/core/variable.py:1130: RuntimeWarning: invalid value encountered in greater
  if not reflexive
//anaconda/lib/python3.5/site-packages/xarray/core/variable.py:1130: RuntimeWarning: invalid value encountered in less
  if not reflexive


In [6]:
# -------------------------------------------------------------------------
# Add necessary support data
# Many other variables are available, but I'm selecting only a small collection of them
CDP['Albedo'] = support['Albedo']
CDP['Tsrf'] = support['Tsrf']
CDP['SP'] = support['SP']
CDP['snowTs'] = support['snowTs']
CDP['snowDepth'] = support['snowDepth']
CDP['groundTs'] = support['groundTs']

# -------------------------------------------------------------------------
# Write netcdf
os.chdir(dirData)
CDP.to_netcdf('CDP.ModTsfc.nc')
print(CDP)

<xarray.Dataset>
Dimensions:     (time: 103995)
Coordinates:
  * time        (time) datetime64[ns] 1993-11-10 1993-11-10T01:00:00 ...
Data variables:
    Tair        (time) float64 0.66 0.11 0.13 -0.52 -0.38 -0.61 -0.67 -0.71 ...
    QS          (time) float64 0.004486 0.004266 0.004317 0.004117 0.004159 ...
    WIND        (time) float64 0.1 0.5 0.4 0.2 0.1 0.1 0.4 0.8 0.3 0.7 1.0 ...
    precipRain  (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    precipSnow  (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    LWdwn       (time) float64 295.8 259.4 265.6 253.6 245.8 225.6 228.9 ...
    SWdwnDir    (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 18.53 ...
    SWdwnDif    (time) float64 0.278 0.0 0.0 0.278 0.0 0.0 0.0 1.944 20.56 ...
    Press       (time) float64 8.726e+04 8.721e+04 8.719e+04 8.719e+04 ...
    SWdwn       (time) float64 0.278 0.0 0.0 0.278 0.0 0.0 0.0 1.944 20.56 ...
    precip      (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0